In [1]:
# %pip freeze > requirements.txt
# %pip install -r requirement.txt

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime

from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import json
import pandas as pd
import pymysql
import re
import requests
import sqlite3
import time
import urllib.parse

# logging.basicConfig(filename='error.log', level=logging.ERROR)

In [3]:
def clean_table_name(jenis, filter_wilayah=''):
    propinsi = filter_wilayah['PROPINSI'].replace(' ','').lower()
    kota = filter_wilayah['KOTA'].replace(' ','').lower()
    kecamatan = filter_wilayah['KECAMATAN'].replace(' ','').lower()
    kelurahan = filter_wilayah['KELURAHAN'].replace(' ','').lower()

    jenis_table = jenis.replace(' ', '')

    if propinsi:
        jenis_table += f'_{propinsi}'
    if kota:
        jenis_table += f'_{kota}'
    if kecamatan:
        jenis_table += f'_{kecamatan}'
    if kelurahan:
        jenis_table += f'_{kelurahan}'
        
    return jenis_table

In [4]:
def db_check(database_type, table_name):
    if database_type.lower() == 'sqlite':
        with sqlite3.connect("./backend/data.db") as connection:
            cursor = connection.cursor()
            tables = {
                f"{table_name}": '"ID" INTEGER PRIMARY KEY NOT NULL, "NAMA" TEXT, "KOORDINAT" TEXT, "ALAMAT" TEXT, "RATING" REAL, "JML_RATING" INTEGER, "TAG_GOOGLE" TEXT, "KELURAHAN" TEXT, "KECAMATAN" TEXT, "KOTA" TEXT, "PROVINSI" TEXT, "TIPE" TEXT, "IDCARI" INTEGER, "DATA_UPDATE" DATETIME',
                "randomized_pos": '"ID" INTEGER PRIMARY KEY NOT NULL, "PROPINSI" TEXT, "KOTA" TEXT, "KECAMATAN" TEXT, "KELURAHAN" TEXT, "KODEPOS" TEXT, "DATA_UPDATE"'
            }
            for table, schema in tables.items():
                cursor.execute(f'CREATE TABLE IF NOT EXISTS {table} ({schema})')

    elif database_type.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                tables = {
                    f'{table_name}': 'ID INT AUTO_INCREMENT PRIMARY KEY, NAMA TEXT, KOORDINAT TEXT, ALAMAT TEXT, RATING FLOAT, JML_RATING INT, TAG_GOOGLE TEXT, KELURAHAN TEXT, KECAMATAN TEXT, KOTA TEXT, PROVINSI TEXT, TIPE TEXT, IDCARI INT, DATA_UPDATE DATETIME',
                    'randomized_pos': 'ID INT AUTO_INCREMENT PRIMARY KEY, PROPINSI TEXT, KOTA TEXT, KECAMATAN TEXT, KELURAHAN TEXT, KODEPOS TEXT, DATA_UPDATE DATETIME'
                }
                for table, schema in tables.items():
                    cursor.execute(f'CREATE TABLE IF NOT EXISTS {table} ({schema})')
            connection.commit()
        finally:
            connection.close()

    else:
        print('Database tidak dikenal')

In [5]:
# df_pos = pd.read_csv('../scrape_kode_pos_indonesia/output/kode_pos.csv')
# df_pos = df_pos.fillna('-')
# df_cari = pd.DataFrame(df_pos['KOTA'].unique(), columns=['KOTA'])

# cek database, kalau kosong isi randomized
def random_pos_check(database_type):
    df_cari = pd.read_csv('../scrape_kode_pos_indonesia/output/kode_pos.csv', dtype=str)
    df_cari = df_cari.sample(frac=1).reset_index(drop=True) # randomized order
    df_cari.fillna('', inplace=True)

    values = []
    for i in range (len(df_cari)):
        propinsi = df_cari.iloc[i]['PROPINSI']
        kota = df_cari.iloc[i]['KOTA']
        kecamatan = df_cari.iloc[i]['KECAMATAN']
        kelurahan = df_cari.iloc[i]['KELURAHAN']
        kodepos = df_cari.iloc[i]['KODE POS']
        update_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        values.append((propinsi, kota, kecamatan, kelurahan, kodepos, update_time))

    if database_type.lower() == 'sqlite':
        with sqlite3.connect('./backend/data.db') as connection:
            cursor = connection.cursor()
            cursor.execute('SELECT COUNT(*) FROM randomized_pos')
            count = cursor.fetchone()[0]
            if count == 0:
                query = ('INSERT INTO randomized_pos (PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?)')
                cursor.executemany(query, values)

    elif database_type.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                cursor.execute('SELECT COUNT(*) FROM randomized_pos')
                count = cursor.fetchone()['COUNT(*)']
                if count == 0:                    
                    query = ('INSERT INTO randomized_pos (PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, DATA_UPDATE) VALUES (%s, %s, %s, %s, %s, %s)')                 
                    cursor.executemany(query, values)
                
            connection.commit()

        finally:
            connection.close()

    else:
        print('Database tidak dikenal')

In [6]:
def create_new_df_cari(database_type, jenis, filter_wilayah=''):
    table_name = clean_table_name(jenis, filter_wilayah)
    
    if database_type.lower() == 'sqlite':
        try:
            with sqlite3.connect('backend/data.db') as connection:
                cursor = connection.cursor()
                cursor.execute(f'SELECT IDCARI FROM {table_name} ORDER BY ID DESC LIMIT 1')
                last_cari = cursor.fetchone()[0]
        except Exception:
            last_cari = 0

        query = f'SELECT PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, ID AS IDCARI FROM randomized_pos WHERE IDCARI > {last_cari}'
        if filter_wilayah:
            propinsi = filter_wilayah['PROPINSI']
            kota = filter_wilayah['KOTA']
            kecamatan = filter_wilayah['KECAMATAN']
            kelurahan = filter_wilayah['KELURAHAN']

        if propinsi:
            query += f' AND PROPINSI = "{propinsi}"'
        if kota:
            query += f' AND KOTA = "{kota}"'
        if kecamatan:
            query += f' AND KECAMATAN = "{kecamatan}"'
        if kelurahan:
            query += f' AND KELURAHAN = "{kelurahan}"'

        with sqlite3.connect('backend/data.db') as connection:
            df_cari = pd.DataFrame(pd.read_sql_query(query, connection))
        
        return df_cari
    
    elif database_type.lower() == 'mariadb':
        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

        try:
            with connection.cursor() as cursor:
                try:
                    cursor.execute(f'SELECT IDCARI FROM {table_name} ORDER BY ID DESC LIMIT 1')
                    last_cari = cursor.fetchone()[0]
                except Exception:
                    last_cari = 0

                query = f'SELECT PROPINSI, KOTA, KECAMATAN, KELURAHAN, KODEPOS, ID AS IDCARI FROM randomized_pos WHERE ID > {last_cari}'
                if filter_wilayah:
                    propinsi = filter_wilayah['PROPINSI']
                    kota = filter_wilayah['KOTA']
                    kecamatan = filter_wilayah['KECAMATAN']
                    kelurahan = filter_wilayah['KELURAHAN']

                if propinsi:
                    query += f' AND PROPINSI = "{propinsi}"'
                if kota:
                    query += f' AND KOTA = "{kota}"'
                if kecamatan:
                    query += f' AND KECAMATAN = "{kecamatan}"'
                if kelurahan:
                    query += f' AND KELURAHAN = "{kelurahan}"'

                cursor.execute(query)
                rows = cursor.fetchall()
                df_cari = pd.DataFrame(rows)
        except Exception as e:
            print(e)

        finally:
            connection.close()
            
        return df_cari

In [7]:
def remove_spaces(input_string):
    result_string = input_string.replace(" ", "")
    return result_string

def create_search_link(query: str, lang, geo_coordinates, zoom):
    if geo_coordinates is None and zoom is not None:
        raise ValueError("geo_coordinates must be provided along with zoom")

    endpoint = urllib.parse.quote_plus(query)

    params = {'authuser': '0',
              'hl': lang,
              'entry': 'ttu',} if lang is not None else {'authuser': '0',
                                                         'entry': 'ttu',}
    
    geo_str = ''
    if geo_coordinates is not None:
        geo_coordinates = remove_spaces(geo_coordinates)
        if zoom is not None:
            geo_str = f'/@{geo_coordinates},{zoom}z'
        else:
            geo_str = f'/@{geo_coordinates}'

    url = f'https://www.google.com/maps/search/{endpoint}'
    if geo_str:
        url += geo_str
    url += f'?{urllib.parse.urlencode(params)}'

    return url

In [8]:
def proxy_auth(proxy_name):
    user, password, domain = [i.replace(' ','') for i in open(f'authentication/{proxy_name}', 'r').read().split(',')]
    return user, password, domain

In [9]:
def map_scraper(database_type, jenis, filter_wilayah, proxy=''):
    if proxy:
        user, password, domain = proxy_auth('proxyscrape')
        proxy_insert = f"{user}:{password}@{domain}"
        proxy_detail = {
                "https":f"http://{proxy_insert}"
            }

    df_cari = create_new_df_cari(database_type, jenis, filter_wilayah)

    query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'

    for i in range(len(df_cari)):
        start_time = time.time()
        dbtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        propinsi = df_cari.iloc[i].iloc[0]
        kota = df_cari.iloc[i].iloc[1]
        kecamatan = df_cari.iloc[i].iloc[2]
        kelurahan = df_cari.iloc[i].iloc[3]
        idcari = int(df_cari.iloc[i].iloc[5])
        cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {propinsi}'
        url_cari = create_search_link(cari, None, '', 18)
    
        if proxy:
            retry_count = 0
            while retry_count <= 10:
                try:
                    response = requests.get(url_cari, proxies=proxy_detail)
                    if response.status_code == 200:
                        break
                except Exception as e:
                    print(e)
                    print('Proxy gagal, mencoba proxy lain')
                    pass
        
                retry_count += 1
            
            if retry_count > 60:
                print('Seluruh proxy gagal')
                break
        else:
            response = requests.get(url_cari)
    
        data_cari = response.text
        soup_cari = BeautifulSoup(data_cari, 'html.parser')
        scripts = soup_cari.find_all('script')
        values = []
    
        for script in scripts:
            if 'window.APP_INITIALIZATION_STATE' in str(script):
                data = str(script).split('=',3)[3]
                data2 = data.rsplit(';',10)[0]
                json_data = json.loads(data2)
                usaha = json_data[3][2][5:]
                json_usaha = json.loads(usaha)
                a = 1
                while True:
                    try:
                        nama = json_usaha[0][1][a][-1][11]
                        koordinat = ', '.join(list(map(str, json_usaha[0][1][a][-1][9][-2:])))
                        alamat = ', '.join(json_usaha[0][1][a][-1][2])
                        tag_google = ', '.join(json_usaha[0][1][a][-1][13])

                        try:
                            rating = float(json_usaha[0][1][a][-1][4][-2])
                        except:
                            rating = 0 

                        try:
                            jml_rating = int(json_usaha[0][1][a][-1][4][-1])
                        except:
                            jml_rating = 0
                            
                        values.append((nama, koordinat, alamat, rating, jml_rating, tag_google, kelurahan, kecamatan, kota, propinsi, jenis, idcari, dbtime))
                        a += 1

                    except:
                        break
            break
        
        if values:
            if database_type.lower() == 'sqlite':
                query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                with sqlite3.connect('backend/data.db') as connection:
                    cursor = connection.cursor()
                    cursor.executemany(query, values)

            elif database_type.lower() == 'mariadb':
                query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
                connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

                try:
                    with connection.cursor() as cursor:
                        cursor = connection.cursor()
                        cursor.executemany(query, values)
                        connection.commit()
                except Exception as e:
                    print(e)
                finally:
                    connection.close()                     

            else:
                print('Database tidak dikenal')
                                        
        print(f'Query {i+1}/{len(df_cari)} {jenis} di kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi} selesai diinput sebanyak {a-1} data')
        print(f'Total waktu query {time.time() - start_time}')
    
    print(f'Scrape {jenis} selesai')

In [10]:
def get_driver(proxy=None):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    
    if proxy:
        user, password, domain = proxy_auth('proxyscrape')
        proxy_insert = f"{user}:{password}@{domain}"

    prox = Proxy()
    prox.proxy_type = ProxyType.MANUAL
    prox.ssl_proxy = f"http://{proxy_insert}"
    capabilities = webdriver.DesiredCapabilities.CHROME
    prox.add_to_capabilities(capabilities)

    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options, desired_capabilities=capabilities)
    except Exception:
        driver = webdriver.Chrome(service=Service('driver/124.0.6367.207/chromedriver-win32/chromedriver.exe'), options=chrome_options, desired_capabilities=capabilities)

    return driver

In [11]:
def map_scraper_with_scrolls(database_type, jenis, filter_wilayah, proxy):
    proxy_count = 0
    cek_proxy = ''

    while proxy_count < 10:
        df_cari = create_new_df_cari(database_type, jenis, filter_wilayah)
        print(f'Ekspektasi jumlah query di cycle ini: {len(df_cari)}')

        driver = get_driver(proxy)

        try:
            for i in range(len(df_cari)):
                start_time = time.time()
                dbtime= datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                propinsi = df_cari.iloc[i].iloc[0]
                kota = df_cari.iloc[i].iloc[1]
                kecamatan = df_cari.iloc[i].iloc[2]
                kelurahan = df_cari.iloc[i].iloc[3]
                idcari = int(df_cari.iloc[i].iloc[5])
                cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {propinsi}'
                url_cari = create_search_link(cari, None, '', 18)
                
                driver.get(url_cari)

                try:
                    WebDriverWait(driver, 10).until(EC.title_contains("Google Maps"))
                    cek_proxy = ''
                except Exception:
                    cek_proxy = 'Proxy gagal'
                    break

                try:
                    divSideBar=driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
                except Exception:
                    print(f'Query {i+1}/{len(df_cari)} kosong kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi}')
                    print(f'Total waktu {time.time() - start_time}')
                    continue

                keepScrolling=True
                while keepScrolling:
                    divSideBar.send_keys(Keys.PAGE_DOWN)
                    div_html = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')

                    if "You've reached the end of the list." in div_html or 'Anda telah mencapai akhir daftar.' in div_html:
                        keepScrolling=False

                soup_cari = BeautifulSoup(driver.page_source, 'html.parser')
                targets = soup_cari.find("div", {'role': 'feed'}).find_all('div', {'class': False})[:-1]
                targets_no_ad = [div for div in targets if div.find('div', {'jsaction':True})]

                values = []
                a = 0
                while True:
                    try:
                        nama = targets_no_ad[a].find_all("div", {'class':True})[0].find('a')['aria-label']
                        coordinate = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', targets_no_ad[a].find_all("div")[0].find("a")['href'])
                        longlat = f'{coordinate.group(1)}, {coordinate.group(2)}'
                        alamat = targets_no_ad[a].find_all('span', {'aria-hidden':'', 'aria-label':'', 'class':''})[3].text.strip()
                        rating = 0

                        try:
                            jml_rating = int(targets_no_ad[a].find_all("div")[17].find_all("span")[4].text.strip()[1:-1].replace(',',''))
                        except:
                            jml_rating = 0

                        try:
                            tag_google = [span for span in targets_no_ad[a].find_all('span', {'aria-label':'', 'aria-hidden':'', 'class':''}) if not span.find('span')][0].text.strip()
                        except:
                            tag_google = ''

                        values.append((nama, longlat, alamat, rating, jml_rating, tag_google, kelurahan, kecamatan, kota, propinsi, jenis, idcari, dbtime))
                        a += 1
                    except Exception:
                        break

                if values:
                    if database_type.lower() == 'sqlite':
                        query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                        with sqlite3.connect('backend/data.db') as connection:
                            cursor = connection.cursor()
                            cursor.executemany(query, values)
        
                    elif database_type.lower() == 'mariadb':
                        query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
                        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
        
                        try:
                            with connection.cursor() as cursor:
                                cursor = connection.cursor()
                                cursor.executemany(query, values)
                                connection.commit()
                        except Exception as e:
                            print(e)
                        finally:
                            connection.close()                     
        
                    else:
                        print('Database tidak dikenal')

                print(f'Query {i+1}/{len(df_cari)} {jenis} di kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi} selesai diinput sebanyak {a-1} data')
                print(f'Total waktu {time.time() - start_time}')
        
        finally:
            proxy_count += 1
            try:
                driver.close()
            except Exception:
                pass

        if i+1 == len(df_cari):
            break

    if proxy_count > 10 and cek_proxy == 'Proxy gagal':    
        status = 'Seluruh proxy gagal'
        print(status)

    status = f'Scrape {jenis} selesai'
        
    print(status)

In [12]:
def map_scraper_with_scrolls_deep(database_type, jenis, filter_wilayah, proxy):
    proxy_count = 0
    cek_proxy = ''

    while proxy_count < 10:
        df_cari = create_new_df_cari(database_type, jenis, filter_wilayah)
        print(f'Ekspektasi jumlah query di cycle ini: {len(df_cari)}')

        driver = get_driver(proxy)

        try:
            for i in range(len(df_cari)):
                start_time = time.time()
                dbtime= datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                propinsi = df_cari.iloc[i].iloc[0]
                kota = df_cari.iloc[i].iloc[1]
                kecamatan = df_cari.iloc[i].iloc[2]
                kelurahan = df_cari.iloc[i].iloc[3]
                idcari = int(df_cari.iloc[i].iloc[5])
                cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {propinsi}'
                url_cari = create_search_link(cari, None, '', 18)
                
                driver.get(url_cari)

                try:
                    WebDriverWait(driver, 10).until(EC.title_contains("Google Maps"))
                    cek_proxy = ''
                except Exception:
                    cek_proxy = 'Proxy gagal'
                    break

                try:
                    divSideBar=driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
                except Exception:
                    print(f'Query {i+1}/{len(df_cari)} kosong kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi}')
                    print(f'Total waktu {time.time() - start_time}')
                    continue

                keepScrolling=True
                while keepScrolling:
                    divSideBar.send_keys(Keys.PAGE_DOWN)
                    div_html = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')

                    if "You've reached the end of the list." in div_html or 'Anda telah mencapai akhir daftar.' in div_html:
                        keepScrolling=False

                soup_cari = BeautifulSoup(driver.page_source, 'html.parser')
                targets = soup_cari.find("div", {'role': 'feed'}).find_all('div', {'class': False})[:-1]
                targets_no_ad = [div for div in targets if div.find('div', {'jsaction':True})]

                values = []
                a = 0
                while True:
                    try:
                        url_cari = targets_no_ad[a].find_all('a')[0]['href']

                        if proxy:
                            user, password, domain = proxy_auth('proxyscrape')
                            proxy_insert = f"{user}:{password}@{domain}"
                            proxy_detail = {
                                    "https":f"http://{proxy_insert}"
                                }
                            response = requests.get(url_cari, proxies=proxy_detail)
                        else:
                            response = requests.get(url_cari)

                        data_cari = response.text
                        soup_cari = BeautifulSoup(data_cari, 'html.parser')
                        scripts = soup_cari.find_all('script')

                        for script in scripts:
                            if 'window.APP_INITIALIZATION_STATE' in str(script):
                                data = str(script).split('=',3)[3]
                                data2 = data.rsplit(';',10)[0]
                                json_data = json.loads(data2)
                                usaha = json_data[3][2][5:]
                                json_usaha = json.loads(usaha)
                                break

                        nama = json_usaha[0][1][1][-1][11]
                        alamat = ', '.join(json_usaha[0][1][1][-1][2])
                        rating = float(json_usaha[0][1][1][-1][4][-4])
                        jml_rating = int(json_usaha[0][1][1][-1][4][-3])
                        longlat = ', '.join(str(k) for k in json_usaha[0][1][1][-1][9][-2:])
                        tag_google = ', '.join(json_usaha[0][1][1][-1][13])

                        values.append((nama, longlat, alamat, rating, jml_rating, tag_google, kelurahan, kecamatan, kota, propinsi, jenis, idcari, dbtime))
                        a += 1
                    except Exception:
                        break

                if values:
                    if database_type.lower() == 'sqlite':
                        query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                        with sqlite3.connect('backend/data.db') as connection:
                            cursor = connection.cursor()
                            cursor.executemany(query, values)
        
                    elif database_type.lower() == 'mariadb':
                        query = f'INSERT INTO {clean_table_name(jenis, filter_wilayah)} (NAMA, KOORDINAT, ALAMAT, RATING, JML_RATING, TAG_GOOGLE, KELURAHAN, KECAMATAN, KOTA, PROVINSI, TIPE, IDCARI, DATA_UPDATE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                        host, port, user, password, database = [i.replace(' ','') for i in open('authentication/mariadb', 'r').read().split(',')]
                        connection = pymysql.connect(host=host, port=int(port), user=user, password=password, database=database, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
        
                        try:
                            with connection.cursor() as cursor:
                                cursor = connection.cursor()
                                cursor.executemany(query, values)
                                connection.commit()
                        except Exception as e:
                            print(e)
                        finally:
                            connection.close()                     
        
                    else:
                        print('Database tidak dikenal')

                print(f'Query {i+1}/{len(df_cari)} {jenis} di kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi} selesai diinput sebanyak {a-1} data')
                print(f'Total waktu {time.time() - start_time}')
        
        finally:
            proxy_count += 1
            try:
                driver.close()
            except Exception:
                pass

        if i+1 == len(df_cari):
            break

    if proxy_count > 10 and cek_proxy == 'Proxy gagal':    
        status = 'Seluruh proxy gagal'
        print(status)

    status = f'Scrape {jenis} selesai'
        
    print(status)

In [13]:
# TODO buat database jenis

In [232]:
# TODO loop keseluruhan kode berdasarkan database jenis

database = 'mariadb' # opsi sqlite(lokal) atau mariadb(server)
proxy = 'proxyscrape' # opsi kosong atau proxyscrape

jenis = 'company registry'

filter_wilayah = {'PROPINSI': 'DAERAH ISTIMEWA YOGYAKARTA',
                  'KOTA': '',
                  'KECAMATAN': '',
                  'KELURAHAN': ''}

db_check(database, clean_table_name(jenis, filter_wilayah))
random_pos_check(database)

# PILIH SALAH SATU
# map_scraper(database, query, filter_wilayah, proxy)
# map_scraper_with_scrolls(database, jenis, filter_wilayah, proxy)
map_scraper_with_scrolls_deep(database, jenis, filter_wilayah, proxy)

Ekspektasi jumlah query di cycle ini: 426
Query 1/426 company registry di kelurahan GIRIPURWO kecamatan GIRIMULYO kota KAB. KULON PROGO provinsi DAERAH ISTIMEWA YOGYAKARTA selesai diinput sebanyak 9 data
Total waktu 10.511500597000122
Query 2/426 company registry di kelurahan WILADEG kecamatan KARANGMOJO kota KAB. GUNUNGKIDUL provinsi DAERAH ISTIMEWA YOGYAKARTA selesai diinput sebanyak 1 data
Total waktu 11.214446783065796
Query 3/426 kosong kelurahan GIRIPENI kecamatan WATES kota KAB. KULON PROGO provinsi DAERAH ISTIMEWA YOGYAKARTA
Total waktu 3.9335215091705322
Query 4/426 company registry di kelurahan TRIHARJO kecamatan SLEMAN kota KAB. SLEMAN provinsi DAERAH ISTIMEWA YOGYAKARTA selesai diinput sebanyak 2 data
Total waktu 7.254425525665283
Query 5/426 company registry di kelurahan UMBULREJO kecamatan PONJONG kota KAB. GUNUNGKIDUL provinsi DAERAH ISTIMEWA YOGYAKARTA selesai diinput sebanyak 5 data
Total waktu 3.272684335708618
Query 6/426 company registry di kelurahan PURWOSARI kecam

KeyboardInterrupt: 

In [ ]:
# TODO buat async function agar bisa beberapa scraper sekaligus
    # TODO ubah cek iterasi dari id cari ke kolom penanda iterasi

In [341]:
database_type = 'mariadb' # opsi sqlite(lokal) atau mariadb(server)
proxy = 'proxyscrape' # opsi kosong atau proxyscrape

jenis = 'restaurant'

filter_wilayah = {'PROPINSI': '',
                  'KOTA': '',
                  'KECAMATAN': 'PANCORAN',
                  'KELURAHAN': ''}

db_check(database_type, clean_table_name(jenis, filter_wilayah))
random_pos_check(database_type)

df_cari = create_new_df_cari(database_type, jenis, filter_wilayah)
print(f'Ekspektasi jumlah query di cycle ini: {len(df_cari)}')

driver = get_driver(proxy)

try:
    for i in range(3):
        start_time = time.time()
        dbtime= datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        propinsi = df_cari.iloc[i].iloc[0]
        kota = df_cari.iloc[i].iloc[1]
        kecamatan = df_cari.iloc[i].iloc[2]
        kelurahan = df_cari.iloc[i].iloc[3]
        idcari = int(df_cari.iloc[i].iloc[5])
        cari = f'{jenis} in {kelurahan}, {kecamatan}, {kota}, {propinsi}'
        url_cari = create_search_link(cari, None, '', 18)
                
        driver.get(url_cari)

        try:
            WebDriverWait(driver, 10).until(EC.title_contains("Google Maps"))
            cek_proxy = ''
        except Exception:
            cek_proxy = 'Proxy gagal'
            break

        try:
            divSideBar=driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
        except Exception:
            print(f'Query {i+1}/{len(df_cari)} kosong kelurahan {kelurahan} kecamatan {kecamatan} kota {kota} provinsi {propinsi}')
            print(f'Total waktu {time.time() - start_time}')
            continue

        keepScrolling=True
        while keepScrolling:
            divSideBar.send_keys(Keys.PAGE_DOWN)
            div_html = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')

            if "You've reached the end of the list." in div_html or 'Anda telah mencapai akhir daftar.' in div_html:
                keepScrolling=False

        soup_cari = BeautifulSoup(driver.page_source, 'html.parser')
        targets = soup_cari.find("div", {'role': 'feed'}).find_all('div', {'class': False})[:-1]
        targets_no_ad = [div for div in targets if div.find('div', {'jsaction':True})]

        values = []
        a = 0
        while True:
            try:
                nama = targets_no_ad[a].find_all("div", {'class':True})[0].find('a')['aria-label']
                coordinate = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', targets_no_ad[a].find_all("div")[0].find("a")['href'])
                longlat = f'{coordinate.group(1)}, {coordinate.group(2)}'
                alamat = targets_no_ad[a].find_all('span', {'aria-hidden':'', 'aria-label':'', 'class':''})[3].text.strip()
                rating = 0

                try:
                    jml_rating = int(targets_no_ad[a].find_all("div")[17].find_all("span")[4].text.strip()[1:-1].replace(',',''))
                except:
                    jml_rating = 0

                try:
                    tag_google = [span for span in targets_no_ad[a].find_all('span', {'aria-label':'', 'aria-hidden':'', 'class':''}) if not span.find('span')][0].text.strip()
                except:
                    tag_google = ''

                values.append((nama, longlat, alamat, rating, jml_rating, tag_google, kelurahan, kecamatan, kota, propinsi, jenis, idcari, dbtime))
                a += 1
            except Exception:
                pass

except Exception:
    pass

Ekspektasi jumlah query di cycle ini: 6


KeyboardInterrupt: 

In [481]:
url = targets_no_ad[9].find_all('a')[0]['href']

user, password, domain = proxy_auth('proxyscrape')
proxy_insert = f"{user}:{password}@{domain}"
proxy_detail = {
        "https":f"http://{proxy_insert}"
    }
response = requests.get(url_cari, proxies=proxy_detail)
data_cari = response.text
soup_cari = BeautifulSoup(data_cari, 'html.parser')
scripts = soup_cari.find_all('script')

In [482]:
for script in scripts:
    if 'window.APP_INITIALIZATION_STATE' in str(script):
        data = str(script).split('=',3)[3]
        data2 = data.rsplit(';',10)[0]
        json_data = json.loads(data2)
        usaha = json_data[3][2][5:]
        json_usaha = json.loads(usaha)

nama = json_usaha[0][1][1][-1][11]

In [495]:
# in-depth
nama = json_usaha[0][1][1][-1][11]
alamat = ', '.join(json_usaha[0][1][1][-1][2])
rating = float(json_usaha[0][1][1][-1][4][-4])
jml_rating = int(json_usaha[0][1][1][-1][4][-3])
ordinat = ', '.join(str(k) for k in json_usaha[0][1][1][-1][9][-2:])
tag = ', '.join(json_usaha[0][1][1][-1][13])

In [548]:
i = 0

In [728]:
scripts[0].getText().split(';')[5]

'window.APP_INITIALIZATION_STATE=[[[7932.073414149613,106.84892605,-6.258879899999999],[0,0,0],[1024,768],13.1],[[["m",[16,52213,33905],13,[693439070,693439070,693439057,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,

In [751]:
scripts[0].getText().split(';')[5].split('=',3)[-1][10728]

'"'

In [792]:
scripts[0].getText().split(';')[5].split('=',3)[-1][6972]

')'

In [798]:
json.loads(split_string_by_length(scripts[0].getText().split(';')[5].split('=',3)[-1],6971)[0])

JSONDecodeError: Expecting value: line 1 column 6972 (char 6971)

['[[[7932.073414149613',
 '106.84892605',
 '-6.258879899999999]',
 '[0',
 '0',
 '0]',
 '[1024',
 '768]',
 '13.1]',
 '[[["m"',
 '[16',
 '52213',
 '33905]',
 '13',
 '[693439070',
 '693439070',
 '693439057',
 '693439057',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439079',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439079',
 '693439079',
 '693439079',
 '693439057',
 '693439057',
 '693439079',
 '693439079',
 '693439079',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439057',
 '693439079',
 '693439079',
 '693439079',
 '693439057',
 '693439057',
 '693439057',
 '693439057

In [804]:
json.loads('"[[[7932.073414149613,106.84892605,-6.258879899999999],[0,0,0],[1024,768],13.1],[[[\"m\",[16,52213,33905],13,[693439070,693439070,693439057,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439070,693439070,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439079]],[\"m\",[15,26106,16952],7,[693439070,693439070,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439079,693439079,693439079,693439079,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439057,693439079,693439079,693439057,693439057,693439057,693439057,693439057,693439079,693439079]],[\"m\",[17,104432,67814],13,[693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439091,693439091,693439091,693439079,693439079,693439070,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439091,693439091,693439091,693439079,693439079,693439070,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439070,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439079,693439070,693439079,693439079,693439079,693439079]]]],[[[2,\"categorical-search-results-injection\"]],null,null,[[null,\"restaurant in KALIBATA, PANCORAN, KOTA ADM. JAKARTA SELATAN, DKI JAKARTA\",null,null,[[[\"3344472147336315477\",\"1539239586200135410\"],\"/g/11dymbvwjh\",null,[4232404341,1068519027],null,null,null,null,null,null,null,null,null,null,\"gcid:chinese_restaurant\"],[[\"3344471807067787805\",\"468368151265539645\"],\"/g/11gh4ly97z\",null,[4232395225,1068520933],null,null,null,null,null,null,null,null,null,null,\"gcid:chicken_wings_restaurant\"],[[\"3344470635873960257\",\"11367972804444266799\"],\"/g/1yg4fd1hx\",null,[4232401728,1068517870],null,null,null,null,null,null,null,null,null,null,\"gcid:japanese_restaurant\"],[[\"3344472146893627139\",\"10413563761834854034\"],\"/g/1ptw3cklw\",null,[4232401625,1068526823],null,null,null,null,null,null,null,null,null,null,\"gcid:seafood_restaurant\"],[[\"3344472440148766573\",\"8586338948083240906\"],\"/g/11gk8rnsg6\",null,[4232400752,1068516826],null,null,null,null,null,null,null,null,null,null,\"gcid:chinese_restaurant\"],[[\"3344470598617130741\",\"18200069043070218828\"],\"/g/1pzsvnggs\",null,[4232342531,1068437903],null,null,null,null,null,null,null,null,null,null,\"gcid:noodle_shop\"],[[\"3344472156814093813\",\"15784692227872866281\"],\"/g/1tcthxd1\",null,[4232413244,1068483499],null,null,null,null,null,null,null,null,null,null,\"gcid:east_javanese_restaurant\"],[[\"3344472148179948651\",\"17354912930314335485\"],\"/g/1hc7bkz1t\",null,[4232402304,1068530836],null,null,null,null,null,null,null,null,null,null,\"gcid:seafood_restaurant\"],[[\"3344471738719630749\",\"16024206350282097467\"],\"/g/11rdg83tg2\",null,[4232364883,1068418580],null,null,null,null,null,null,null,null,null,null,\"gcid:coffee_shop\"],[[\"3344472147040103661\",\"15865665419761145946\"],\"/g/11bw5vqs6p\",null,[4232399246,1068528698],null,null,null,null,null,null,null,null,null,null,\"gcid:javanese_restaurant\"],[[\"3344472147247596527\",\"5834992900117684579\"],\"/g/1ptxfykl0\",null,[4232401206,1068520610],null,null,null,null,null,null,null,null,null,null,\"gcid:chinese_noodle_restaurant\"],[[\"3344472337177880389\",\"624372610148166360\"],\"/g/11rxwlr6r3\",null,[4232403411,1068520088],null,null,null,null,null,null,null,null,null,null,\"gcid:restaurant\"],[[\"3344472123399794357\",\"9988794004992612846\"],\"/g/1txd6mql\",null,[4232394300,1068559941],null,null,null,null,null,null,null,null,null,null,\"gcid:restaurant\"],[[\"3344470606880622999\",\"13456799594155730620\"],\"/g/11ckqr9gjw\",null,[4232397338,1068558673],null,null,null,null,null,null,null,null,null,null,\"gcid:indonesian_restaurant\"],[[\"3344472147027353097\",\"17814228135795999602\"],\"/g/11cjkw44y6\",null,[4232395606,1068527302],null,null,null,null,null,null,null,null,null,null,\"gcid:noodle_shop\"],[[\"3344472147148326003\",\"7561968935800418763\"],\"/g/11fylw48x3\",null,[4232397402,1068520883],null,null,null,null,null,null,null,null,null,null,\"gcid:restaurant\"],[[\"3344472123387255411\",\"10635858467581436157\"],\"/g/11bwkgk80h\",null,[4232399136,1068557629],null,null,null,null,null,null,null,null,null,null,\"gcid:asian_restaurant\"],[[\"3344471616515390383\",\"16126013172835428490\"],\"/g/11kv82m3bc\",null,[4232414463,1068458879],null,null,null,null,null,null,null,null,null,null,\"gcid:restaurant\"],[[\"3344472147274295111\",\"14864347072191842064\"],\"/g/11gcj5zb_7\",null,[4232404146,1068523260],null,null,null,null,null,null,null,null,null,null,\"gcid:cafe\"],[[\"3344471607199492923\",\"7874873901882901148\"],\"/g/11h_kf3y50\",null,[4232391457,1068527283],null,null,null,null,null,null,null,null,null,null,\"gcid:sundanese_restaurant\"]],5,null,1,null,null,null,null,[[[[\"restaurant\",null,0.42946136],[\"id_subdistrict\",null,0.4179249],[\"political\",null,0.32062677],[\"administrative_area3\",null,0.3134437],[\"administrative_area\",null,0.312573],[\"administrative_area4\",null,0.26490343]]],0.42946136,0.1295918],null,null,null,1]]],[null,\""')

JSONDecodeError: Extra data: line 1 column 85 (char 84)